In [ ]:
'''
SAM GREEN'S CODE
'''

# import xarray as xr
# import re
# import pandas as pd
# import numpy as np
# import rioxarray as rxr
# import glob
# import os

# # Plotting stuff:
# import matplotlib.pyplot as plt
# from matplotlib import colormaps
# import matplotlib.colors as mcolors
# from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
# import matplotlib.cm as cm
# import cartopy.crs as ccrs
# import cartopy.feature as cft
# import cmocean as cmo
# import matplotlib as mpl
# import matplotlib.ticker as mticker

# import dask
# from dask.distributed import Client

# client = Client()
# client

# # Function to extract timestamp from filename
# def extract_timestamp(filename):
#     match = re.search(r"_(\d{8}T\d{6})Z", filename)
#     if match:
#         timestamp = pd.to_datetime(match.group(1), format="%Y%m%dT%H%M%S")
#         return timestamp.replace(second=0, microsecond=0)
#     return None


# def open_and_add_time(file):
#     #O\open files
#     ds = rxr.open_rasterio(file, chunks={"x": 1024, "y": 1024}, masked=True)[0]
#     #reproject to lat/lon
#     da = ds.rio.reproject('epsg:4326')
#     timestamp = extract_timestamp(os.path.basename(file))

#     # Add time dimension
#     da = da.expand_dims(time=[timestamp])  # Add time as a new dimension

#     return da

# file_list = sorted(glob.glob("/g/data/rv74/satellite-products/arc/der/himawari-ahi/cloud/ct/latest/2022/01/01/S_NWC_CT*"))

# # Delayed open and timestamp attach
# delayed_datasets = [dask.delayed(open_and_add_time)(f) for f in file_list]
# # Trigger Dask to load data
# datasets = dask.compute(*delayed_datasets)
# # Concatenate along time
# stacked = xr.concat(datasets, dim="time")
# # rename to lat/lon
# stacked = stacked.rename({"x": "lon", "y": "lat"})
# # slice lat/lon to have the  Australian regional domain
# stacked1 = stacked.sel(lat=slice(19.425, -64.945), lon=slice(65.055, 196.945))
# # set the names for all the cloud types
# comment = '1:  Cloud-free land; 2:  Cloud-free sea; 3:  Snow over land;  4:  Sea ice; 5:  Very low clouds; 6:  Low clouds; 7:  Mid-level clouds;  8:  High opaque clouds; 9:  Very high opaque clouds;  10:  Fractional clouds; 11:  High semitransparent thin clouds;  12:  High semitransparent moderately thick clouds;  13:  High semitransparent thick clouds;  14:  High semitransparent above low or medium clouds;  15:  High semitransparent above snow/ice'

# # Use regex to extract values and labels
# matches = re.findall(r'(\d+):\s+([^;]+)', comment)

# # Convert to dictionary
# category_dict = {int(num): desc.strip() for num, desc in matches}

# # Print result
# for k, v in sorted(category_dict.items())[:14]:
#     print(f"{k}: {v}")

# fig = plt.figure(figsize=(16, 8))

# ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=130))
# ax.coastlines(resolution="50m", color='white')

# # Define the values and corresponding colors from tab20
# values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]  # Adjust based on your dataset
# # Set the min and max colors from tab20
# min_color = mpl.colormaps['tab20'](0 / 20)
# max_color = mpl.colormaps['tab20'](19 / 20)  # Last color of tab20

# # Generate intermediate colors by linearly spacing them within tab20
# num_classes = len(values)
# colors = [mpl.colormaps['tab20'](i / (num_classes - 1)) for i in range(num_classes)]

# # Ensure min/max colors are set explicitly
# colors[0] = min_color
# colors[-1] = max_color

# # Create discrete colormap
# cmap = mcolors.ListedColormap(colors)
# # Define boundaries for normalization (each value gets its own bin)
# boundaries = np.arange(min(values) - 0.5, max(values) + 1.5, 1)  # Adjusted for correct binning
# norm = mcolors.BoundaryNorm(boundaries, cmap.N)

# img = stacked1.sel(time='2022-01-01T03:00:00.000000000').ct.plot(ax=ax, x="lon", y="lat", transform=ccrs.PlateCarree(), cmap=cmap, norm=norm, add_colorbar=False)

# # Add a discrete colorbar
# cbar = fig.colorbar(img, ax=ax, orientation="vertical", fraction=0.03, pad=0.02)
# cbar.set_label("Cloud Type (CT)", fontsize=12)
# cbar.set_ticks(values)  # Ensure ticks match category values
# cbar.set_ticklabels([f"{k}: {v}" for k, v in sorted(category_dict.items())[:14]])  # Custom labels

# # Add gridlines with labels
# gl = ax.gridlines(draw_labels=True, linestyle="--", linewidth=1.0)
# gl.top_labels = False  # Disable top labels
# gl.right_labels = False  # Disable right labels

# # Increase the number of ticks on the x-axis
# gl.xlocator = mticker.FixedLocator(np.arange(70, 190, 20))  # Adjust step size as needed

In [6]:
'''
MATT LIPSON CODE
'''


# import pandas as pd
# import xarray as xr
# import rioxarray as rxr
# import cartopy.crs as ccrs
# import os

# # dates of interest
# sdate = '2020-01-01T00:00'
# edate = '2020-01-01T00:00'
# freq = '10min'

# xmin, xmax, ymin, ymax = 110, 155, -45, -9   # AU domain

# dates = pd.date_range(sdate, edate, freq=freq)

# for date in dates:

#     year, month, day = date.strftime('%Y'), date.strftime('%m'), date.strftime('%d')
#     hour, minute = date.strftime('%H'), date.strftime('%M')

#     fpath = f'/g/data/rv74/satellite-products/arc/der/himawari-ahi/cloud/ct/latest/{year}/{month}/{day}'
#     fname_prefix = 'S_NWC_CT_HIMA08_HIMA-N-NR' # Note that this changes to HIMA09 at some point
#     fname_in = f'{fpath}/{fname_prefix}_{year}{month}{day}T{hour}{minute}00Z.nc'

#     # use rioxarray to open the data with projection information
#     orig = rxr.open_rasterio(fname_in, masked=True)[0]

#     # use rioxarray to reproject the data
#     da = orig['ct'].rio.reproject('epsg:4326')

#     # select a subset based on your desired lat/lon
#     subset = da.sel(y=slice(ymax,ymin),x=slice(xmin,xmax))

#     # plot with cartopy
#     plt.close('all')
#     fig = plt.figure()
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     subset.plot(ax=ax, add_colorbar=True)
    
#     ax.coastlines(resolution='50m',color='r',lw=0.4,zorder=5)
#     # ax.coastlines(resolution='10m',color='r',lw=0.5)
#     gl = ax.gridlines(draw_labels=True)
#     gl.top_labels   = False
#     gl.right_labels = False
#     gl.ylines       = False
#     gl.xlines       = False
#     gl.xlabel_style = {'size': 6}
#     gl.ylabel_style = {'size': 6}
#     ax.set_title(f"Himawari cloud type:\n{date.strftime('%Y-%m-%d %H%M')}Z",fontsize=10,y = 1.02)

#     plt.show()

SyntaxError: incomplete input (3891110700.py, line 1)

In [7]:
# https://scores.readthedocs.io/en/stable/tutorials/Fractions_Skill_Score.html

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pathlib import Path
from datetime import datetime
import re
import pandas as pd
from scores.spatial import fss_2d

from dask.distributed import Client, wait
import os, sys
import psutil

sys.path.append('/home/548/cd3022/repos/Irradiance-comparisons/Irradiance-comparisons')
import logger
from read_datasets import read_dataset

LOG = logger.get_logger(__name__)

In [3]:
client = Client(
    n_workers=24,
    threads_per_worker=1
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 24
Total threads: 24,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33317,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:34217,Total threads: 1
Dashboard: /proxy/36865/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:32855,


# Load Himawari CMIC

In [4]:
ancil = xr.open_dataset('/g/data/ra22/satellite-products/arc/obs/himawari-ahi/fldk/latest/ancillary/00000000000000-P1S-ABOM_GEOM_SENSOR-PRJ_GEOS141_2000-HIMAWARI8-AHI.nc')
ancil = ancil.isel(time=0)

In [5]:
# Define region bounds
lat_min, lat_max = -44.5, -10.0
lon_min, lon_max = 112.0, 156.126

path = Path('/g/data/rv74/satellite-products/arc/der/himawari-ahi/cloud/cmic/latest/2020/01/')
files = sorted(path.rglob("*.nc"))

# Pre-load ancillary coordinates once
lat_ancil = ancil.lat.data
lon_ancil = ancil.lon.data


def make_preprocess_func(filepaths):
    def _preprocess(ds, file_idx):
        f = filepaths[file_idx]
        match = re.search(r"_(\d{8}T\d{6})Z\.nc$", f.name)
        time = pd.to_datetime(match.group(1), format="%Y%m%dT%H%M%S")
        ds = ds.assign_coords(
            time=time,
            lat=(("ny", "nx"), lat_ancil),
            lon=(("ny", "nx"), lon_ancil)
        )
        ds["cmic_lwp"] = ds["cmic_lwp"].fillna(0)
        return ds

    def preprocess_with_index(i, ds):
        return _preprocess(ds, i)

    return preprocess_with_index


# Open all files lazily with Dask
preprocess_func = make_preprocess_func(files)

datasets = []
for i, f in enumerate(files):
    ds = xr.open_dataset(f, chunks="auto")
    ds = preprocess_func(i, ds)
    datasets.append(ds)

ds = xr.concat(datasets, dim="time")

KeyboardInterrupt: 

In [ ]:
# lat_min = -44.5
# lat_max = -10.0
# lon_min = 112.0
# lon_max = 156.126

# path = Path('/g/data/rv74/satellite-products/arc/der/himawari-ahi/cloud/cmic/latest/2020/01/01/')
# files = [f for f in path.rglob('*.nc')]

# def open_cmic(file):
#     # open data file
#     ds = xr.open_dataset(
#         file,
#         chunks='auto'
#     )

#     # time to be assigned
#     match = re.search(r"_(\d{8}T\d{6})Z\.nc$", f.name)
#     time=pd.to_datetime(match.group(1), format="%Y%m%dT%H%M%S")

#     # assign lat/lon (time) coords using ancillary file
#     ds = ds.assign_coords(
#         time=time,
#         lat=(("ny", "nx"), ancil.lat.data),
#         lon=(("ny", "nx"), ancil.lon.data)
#     )
    
#     mask = (
#         (ds.lat >= lat_min) & (ds.lat <= lat_max) &
#         (ds.lon >= lon_min) & (ds.lon <= lon_max)
#     )
#     ds = ds.where(mask, drop=True)
#     return ds


# datasets = []
# for f in (files):
#     ds = open_cmic(f)
#     datasets.append(ds)
# ds = xr.concat(datasets, dim="time")

# ds['cmic_lwp'] = xr.where(ds.cmic_lwp.isnull(), 0, ds.cmic_lwp)

# ds = ds.where((ds.lon >= lon_min) & (ds.lon <= lon_max), drop=True)

In [ ]:
def get_barra(var):
    BARRA = Path('/g/data/ob53/BARRA2/output/reanalysis/')
    BARRA_R2 = BARRA / "AUS-11/BOM/ERA5/historical/hres/BARRA-R2/v1"
    BARRA_R2_DIR = BARRA_R2 / "1hr"
    
    files = sorted([f for f in BARRA_R2_DIR.glob(f'{var}/latest/*1hr_20*.nc')])
    
    def _preprocess(ds):
        return ds.sel(lat=slice(-44.5, -10), lon=slice(112, 156.26))
    barra_r2 = xr.open_mfdataset(
        files,
        chunks={'time':24, 'lat':256, 'lon':256},
        concat_dim='time',
        combine='nested',
        data_vars='minimal',
        coords='minimal',
        compat='override',
        parallel=True,
        preprocess=_preprocess
    )
    return barra_r2

def get_himawari(year):
    # get Himawari data
    himawari_list = []
    for month in range (1, 2):
        date = f'{year}-{month:02d}' 
        himawari = read_dataset(
                dataset='himawari',
                resolution='hourly',
                date=date
            )
        himawari_list.append(himawari)
    him_ds = xr.concat(himawari_list, dim='time', data_vars='minimal')
    return him_ds

In [ ]:
bar = get_barra('clwvi')
bar = bar.sel(time=ds.time, method='nearest')

In [ ]:
import xesmf as xe

# Define target 1D grid
target = xr.Dataset({
    "lat": (["lat"], np.linspace(lat_min, lat_max, bar.lat.shape[0])),
    "lon": (["lon"], np.linspace(lon_min, lon_max, bar.lon.shape[0])),
})

regridder = xe.Regridder(ds, target, "bilinear")
ds_reg = regridder(ds)

ds_reg = ds_reg.interp(
    lat=bar.lat,
    lon=bar.lon,
    method='linear'
)

In [ ]:
# Plot Himawari

fig, ax = plt.subplots(nrows=2, ncols=2, subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(12, 8))

pcm = ax[0,0].pcolormesh(
    ds_reg.lon,
    ds_reg.lat,
    ds_reg.cmic_lwp.mean(dim='time'),
    cmap="viridis",
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=0.2
)

ax[0,1].pcolormesh(
    ds_reg.lon,
    ds_reg.lat,
    xr.where(ds_reg.cmic_lwp.mean(dim='time') > 0.08, 1, 0),
    cmap="Reds",
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=0.2
)

ax[1,0].pcolormesh(
    bar.lon,
    bar.lat,
    bar.clwvi.mean(dim='time'),
    cmap="viridis",
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=0.2
)

ax[1,1].pcolormesh(
    ds_reg.lon,
    ds_reg.lat,
    xr.where(bar.clwvi.mean(dim='time') > 0.08, 1, 0),
    cmap="Reds",
    transform=ccrs.PlateCarree(),
    vmin=0,
    vmax=0.2
)


ax[0,0].set_title('Himawari')
ax[1,0].set_title('BARRA')
ax[0,1].set_title('Himawari > 0.08')
ax[1,1].set_title('BARRA > 0.08')
plt.colorbar(pcm, ax=ax, label="1-day mean LWP (kg m-2)")

for a in ax.flatten():
    a.coastlines()

plt.show()

In [ ]:
obs = ds_reg.cmic_lwp
fcst = bar.clwvi

def drop_duplicate_times(da):
    _, index = np.unique(da['time'], return_index=True)
    return da.isel(time=index)

obs = drop_duplicate_times(obs)
fcst = drop_duplicate_times(fcst)

obs, fcst = xr.align(obs, fcst)

obs = obs.compute()
fcst = fcst.compute()

obs = obs.mean(dim='time')
fcst = fcst.mean(dim='time')

thresholds = np.linspace(0.02, 0.14, 20)
window_sizes = np.arange(1, 21)  # 1..10

# Empty array to store results
fss_results = np.zeros((len(thresholds), len(window_sizes)))
L_vals = np.zeros(len(thresholds))

for i, thr in enumerate(thresholds):
    f = xr.where(obs > thr, 1, 0).sum().item() / obs.size
    L = 0.5 + f/2
    L_vals[i] = L
    for j, win in enumerate(window_sizes):
        fss = fss_2d(
            fcst,
            obs,
            event_threshold=thr,
            window_size=(win, win),
            spatial_dims=["lat", "lon"],
        )
        # Extract a scalar if needed
        # if isinstance(fss, xr.DataArray):
        #     fss = fss.item()
        fss_results[i, j] = fss

fig, ax = plt.subplots(figsize=(7, 5))
im = ax.imshow(
    fss_results,
    origin="lower",
    aspect="auto",
    extent=[window_sizes[0]*12, window_sizes[-1]*12, thresholds[0], thresholds[-1]],
    cmap="viridis",
)
cbar = plt.colorbar(im, ax=ax, label="FSS")

ax.set_xlabel("Window size (km)")
ax.set_ylabel("Threshold (kg m-2)")
ax.set_title("FSS")


# === Overlay hatching for FSS > L for each row ===
# Create coordinate grids matching imshow extent
X, Y = np.meshgrid(window_sizes * 12, thresholds)

# Mask for cells above the row-dependent L
mask = np.zeros_like(fss_results, dtype=bool)
for i in range(len(thresholds)):
    mask[i, :] = fss_results[i, :] < L_vals[i]

# Overlay hatched transparent contour
ax.contourf(
    X,
    Y,
    mask,
    levels=[0.5, 1],
    colors="none",
    hatches=["////"],
    alpha=0,
)


plt.show()